In [1]:
import pandas as pd
import numpy as np
from DS_detector import *
from DS_Planck_Unet import load_planck_model
from tqdm.notebook import tqdm

In [2]:
model = load_planck_model(
    '/home/rt2122/Models/planck_z/f8d2.ep0014-vl0.006570-l0.004067.hdf5')

In [3]:
steps = [2 ** i for i in range(6, 2, -1)]
steps

[64, 32, 16, 8]

In [4]:
thr_list = [0.1, 0.2, 0.9]

In [5]:
step_dict = {step : {} for step in steps}
step_dict

{64: {}, 32: {}, 16: {}, 8: {}}

In [6]:
for step in tqdm(steps):
    all_dict = gen_pics_for_detection(6, model, step=step)
    for thr in tqdm(thr_list):
        coords = detect_clusters_connected(all_dict, thr, 6)
        step_dict[step][thr] = coords
        coords.to_csv(
            '/home/rt2122/Data/step_comp/ep14_thr{thr}_st{st}.csv'.format(thr=thr,
                                                                         st=step))

/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
step_stat = {}

In [12]:
for step in step_dict:
    step_stat[step] = []
    for thr in step_dict[step]:
        df = {}
        for cat in ['planck_z', 'planck_no_z', 'mcxcwp', 'act']:
            cur_cat = step_dict[step][thr]
            cur_cat = cur_cat[cur_cat['catalog'] == cat]
            if len(cur_cat) == 0:
                continue
            recall = np.count_nonzero(cur_cat['status'] == 'tp') / len(cur_cat)
            df[cat] = recall
        df['fp'] = np.count_nonzero(step_dict[step][thr]['status'] == 'fp')
        df['all'] = len(step_dict[step][thr])
        df = pd.DataFrame(df, index=[thr])
        step_stat[step].append(df)
    step_stat[step] = pd.concat(step_stat[step])

In [14]:
for step in step_stat:
    step_stat[step]['step'] = step

In [28]:
thr_stat = {thr:[] for thr in thr_list}
for step in step_stat:
    for thr in thr_list:
        thr_stat[thr].append(step_stat[step][step_stat[step].index == thr])

In [27]:
for step in step_stat:
    step_stat[step]['thr'] = step_stat[step].index

In [29]:
for thr in thr_stat:
    thr_stat[thr] = pd.concat(thr_stat[thr])

In [30]:
thr_stat[0.1]

,planck_z,planck_no_z,mcxcwp,fp,all,step,thr
0.1,0.875,0.666667,0.04,134,242,64,0.1
0.1,0.925,0.722222,0.08,136,244,32,0.1
0.1,0.925,0.833333,0.08,145,253,16,0.1
0.1,0.950,0.777778,0.10,119,227,8,0.1


In [31]:
thr_stat[0.2]

,planck_z,planck_no_z,mcxcwp,fp,all,step,thr
0.2,0.825,0.666667,0.04,84,192,64,0.2
0.2,0.900,0.611111,0.06,74,182,32,0.2
0.2,0.925,0.666667,0.06,67,175,16,0.2
0.2,0.925,0.777778,0.08,60,168,8,0.2


In [32]:
thr_stat[0.9]

,planck_z,planck_no_z,mcxcwp,fp,all,step,thr
0.9,0.500,0.055556,0.0,5,113,64,0.9
0.9,0.500,0.055556,0.0,2,110,32,0.9
0.9,0.425,0.055556,0.0,1,109,16,0.9
0.9,0.450,0.000000,0.0,1,109,8,0.9


In [33]:
steps = [2 ** i for i in range(2, 0, -1)]
steps

[4, 2]

In [34]:
step_dict = {step : {} for step in steps}
step_dict

{4: {}, 2: {}}

In [ ]:
for step in tqdm(steps):
    all_dict = gen_pics_for_detection(6, model, step=step)
    for thr in tqdm(thr_list):
        coords = detect_clusters_connected(all_dict, thr, 6)
        step_dict[step][thr] = coords
        coords.to_csv(
            '/home/rt2122/Data/step_comp/ep14_thr{thr}_st{st}.csv'.format(thr=thr,
                                                                         st=step))